# Student Performance Indicator: Model Training

### About the Dataset
This project understands how the student's performance (test scores) is affected by other variables such as Gender, Ethnicity, Parental level of education, Lunch and Test preparation course.

**Source:** https://www.kaggle.com/datasets/spscientist/students-performance-in-exams?datasetId=74977

### Description
- **gender:** Sex of students
- **race/ethnicity:** Indicates the ethnicity of students
- **parental level of education:** Parents' final education 
- **lunch:** Type of lunch the student had before test
- **test preparation course:** Tells whether the course was completed before the test or not
- **math score:** Math's score of the student
- **reading score:** Reading's score of the student
- **writing score:** Writing's score of the student

### Objective
Our objective is to predict a student's score on the test they underwent.